# Review of Django Model, View, Template Concept

![MVT Diagram](https://res.cloudinary.com/carter3689/image/upload/v1496173347/django_mvt_funlge.png)

## Models

In Django, Models allow us to create classes that will then become our tables in a database. You have your choice of many database backends, but we've been using PostgreSQL. Our models and database are very hand-in-hand and we use migrations to do the heavy SQL syntax lifting for us. 

## Migrations
When we make a migration, we are essentially creating a file that maps our table contents in an Object Oriented way. These files can be found in the `migrations` folder, once you have created a migration of course. **Note **These files can be edited as well . If you somehow make a mistake in a future migration, you can roll back a migration by simply using a previous migration

For example:
    Say I have two migrations (0001-initial.py and 0002-update.py) and I realized that I added a field inside a model that I didn't want. I can roll back to the first migration by giving the `migrate` keyword the name of the previous migration (0001-initial.py in this case). This revert the database back to it's previous state and you can add/remove to it as necessary. 

## Views

Views are what allow us access to the models that we have created in our project. Gathering data from the database happens when we execute what is known in Django as a **QuerySet**. A **QuerySet** can gather ALL (e.g. Bears.objects.all()) of the contents of a specific table column, it can gather the data and filter it (e.g. `Bears.objects.filter(name='Poppa') #We only want to bring back the name of the bear Poppa in this case`) or Order the results in a certain fashion (e.g `Presidents.objects.order_by('first_name') # This gives us back the first_name of our presidents in alphabetical order`)

Views are where most of our python code will be written because this is where a large part of our logic will live. To re-introduce the views concept we will start by talking about the most basic of django views...the function based view.

## Function Based Views

Function Based Views are views that have been written in the "python function" format. These views are the entry point to learning about Django's View system and this is what we created last week with our Polls app (at the beginning). If you recall, these views are a bit longer to write, but give the best understanding of how things should fit together. To recap what a view looks like, let's look at some code:

In [ ]:
# from django.http import HttpResponse
def index(request):
    return HttpResponse("Hi My Name is...")

""" 
This will return to the page the string 'Hi My Name is...' to the front end when a particular url route is given using this view
"""

######## Another Example of a View ############
#from django.shortcuts import render
def index(request):
    webpage_list = AccessRecord.objects.order_by('date')
    date_dict = {'access_records': webpage_list}

        return render(request,'prework_app/index.html',{context=date_dict} )

"""This index view gathers access_record information from my database and displays that to the index.html page
    
    A few things to point out here are:
        the use of the render() method
        what render expects to be passed to it 
"""

In the second view example, we gather our data from our database placing that into a webpage_list variable and then passing that to a dictionary so that when we call render to display our data to our index.html page...we can pass it to the context Keyword argument that expects to receive a dictionary. 

The request passed in, doesn't neccesarily need to be called 'request' persay. This however is the Django naming convention as far as the syntax is concerned. The request parameter when coupled with the render() method attempts to bring back some sort of contextual information so that it can be used by the template afterwards.

This contextual information is what should be passed to 'context' as a dictonary. Once we have the context, we can then display the info on the html with template tags (e.g. {% for in access_records %})

## Class based Views

Class Based Views are the more common of the view types and alot easier to work with mainly because of the way Django Class Based Views inherit from some of the django modules.

The four Classed Based Views (or CBVs) are :
- View
- TemplateView
- DetailView
- ListView

Another nice feature of Class based views is, we can handle HTTP responses in a much cleaner fashion. For example, let's say we wanted to find out if someone is sending a POST request to our database from a form in our project. 

This is what we would have to write for a function based view:

In [ ]:
#from django.http import HttpResponse
def some_view(request):
    if request.method == 'POST':
        #POST something to our DB
        return HttpResponse('Your Request was POSTED')

**If we were using a Class Based View though... **

In [ ]:
#from django.http import HttpResponse
#from django.views import View

class some_view(View):
    def post(self,request):
        #POST something to our DB
        return HttpResponse('Your Request was POSTED')

Though this example has the same amount of lines of code, the modularity here is much different. In the function view, we would have to use a bunch of if/else statements to get our desired result. Plus, this wouldn't really scale well if we wanted to reuse this function for a different outcome (e.g POSTing to a different location).

On the other hand, we can reuse this class based view. This could be reused in a different part of our application much easier because of inheritence. 

With these benefits we also have some extra goodies that come with Class Based Views. These namely come frome the **TemplateView, DetailView, and the ListView**.

#### Here's an example of each

In [ ]:
#TemplateView

class IndexView(TemplateView):
    template_name = 'prework_app/index.html'

    def get_context_data(self, **kwargs):
        context = super().get_context_data(**kwargs)
        context['injectme'] = 'Basic Injection' #name='Joel Carter
        return context
    
    
#HTML

    
#List View
    
class UserView(ListView):
    context_object_name = 'users'
    model = models.Users
    # The ListView by default will create a context_list dictionary which lowercases the model.
    # So when the above Users model comes back to the front end it is callable by a different name
    # That name is users_list
    
# Detail View

class PresidentView(DetailView):
    model = models.Presidents
    
    #The DetailView by default will create context which again lowercases the model.
    #When we want to call this model on the front end, we call it by calling on presidents


## Side Note about *args and **kwargs

To help explain this a bit better I found some more documentation while searching the interwebs of stack overflow.

SO...

The *args allows you to create a function that accepts any number of arguments when you don't **really** know how many the user may pass in. These args get passed in sort of like a tuple.

The **kwargs allows you to let the user specify what they want to pass in and give the items a name. These get passed in as a sort of dictionary.

In [6]:
# Helpful examples

#########*args###########

def print_everything(*args):
        for count, thing in enumerate(args):
         print( '{0}. {1}'.format(count, thing))
        
print_everything('apple', 'banana', 'cabbage')



##########**kwargs##########

def table_things(**kwargs):
     for name, value in kwargs.items():
        print( '{0} = {1}'.format(name, value))
        
        
print("-------------")        
table_things(apple = 'fruit', cabbage = 'vegetable')


0. apple
1. banana
2. cabbage
-------------
apple = fruit
cabbage = vegetable


## URLS

The next critical step to using views is to wire them up to a url. We do this inside of our urls.py file and all of our urls should go inside of the urlpatterns list.

When using class based views, we attach the views to a url in a slightly different way then we would a function based one.

In [ ]:
#Here's how it's done:

#urls.py

#from prework_app import views
urlpatterns = [
    url(r'^$', views.UserView.as_view(template_name='prework_app/index.html'), name='index'),
]

"""This will map to the index page of our app. """

## Templates

From the url, we then get the data that we set up in our view, passed to our url, and the template then renders that and makes it uniform (pretty).

In [ ]:
<!DOCTYPE html>
{% extends "prework_app/base.html" %}
{% load staticfiles %}
  {% block body_block %}
    <h1>Hello how are you</h1>
    <h2>Here's your data</h2>
    {{injectme}}
    <div class="djangotwo">
      {% if users_list %}
        <table>
          <thead>
            <th>Site Name</th>
            <th>Date Access</th>
          </thead>
          {% for user in users_list %}
            <tr>
              <td>{{user.name}}</td>
              <td>{{user.email}}</td>
            </tr>
          {% endfor %}
        </table>
        {% else %}
          <p>No Records Found</p>
        {% endif %}
    </div>
    {% endblock %}